In [14]:
import pandas as pd
import re
import os
import logging
from pathlib import Path

# --- Setup basic logging ---
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [15]:
def natural_sort_key(s):
    """A helper function to allow for natural sorting of column names like 'Class 10' after 'Class 2'."""
    match = re.search(r'Class (\d+)', s)
    return int(match.group(1)) if match else -1

def run_final_grade_update():
    """
    Finds and merges all Canvas and iClicker files, preserving official bCourses
    column names and sorting them naturally before calculating the final total.
    """
    for p in Path.cwd().glob('updated_*'):
        if p.is_file(): p.unlink()
    # --- 1. File Discovery ---
    try:
        all_files_in_dir = os.listdir()
        bc_pattern = '.*_Grades-CIVENG_93.csv'
        ic_pattern = 'iClicker_GradesExport_Canvas_.*.csv'

        bc_filenames = sorted([f for f in all_files_in_dir if re.match(bc_pattern, f)], reverse=True)
        ic_filenames = sorted([f for f in all_files_in_dir if re.match(ic_pattern, f)])

        if not bc_filenames: raise StopIteration("No Canvas gradebook files found.")

        logging.info(f"✅ Found {len(bc_filenames)} Canvas gradebooks and {len(ic_filenames)} iClicker files.")
        
    except StopIteration as e:
        logging.error(f"❌ Critical: Could not find required CSV files. {e}")
        return

    # --- 2. Build a Master Map of Official Column Names ---
    # This is crucial for keeping the bCourses format like 'Class 1 - Poll (12345)'
    poll_column_map = {}
    for filename in bc_filenames:
        try:
            df = pd.read_csv(filename, dtype=str, nrows=0) # Read only headers for efficiency
            for col in df.columns:
                match = re.match(r'(Class \d+ - Poll)( \(\d+\))', col)
                if match:
                    simple_name = match.group(1)
                    poll_column_map[simple_name] = col # Map 'Class 1 - Poll' -> 'Class 1 - Poll (12345)'
        except Exception as e:
            logging.warning(f"Could not read headers from {filename}: {e}")

    logging.info(f"Built a map of {len(poll_column_map)} official poll column names.")
    
    # --- 3. Load and Prepare a Master Gradebook ---
    try:
        master_bc_filename = bc_filenames[0]
        master_df = pd.read_csv(master_bc_filename, dtype=str)

        # Find the official total column name from the main gradebook
        master_total_col = next((col for col in master_df.columns if re.match(r'iClicker \(Total\) \(\d+\)', col)), None)
        if not master_total_col:
            logging.error("❌ Critical: Could not find the 'iClicker (Total) (number)' column.")
            return

        # Separate the master file into its components
        is_student = pd.to_numeric(master_df['SIS User ID'], errors='coerce').notna()
        student_data = master_df[is_student].copy()
        special_rows = master_df[~is_student].copy()
        
        is_test_student = student_data['Student'] == 'Student, Test'
        test_student_row = student_data[is_test_student].copy()
        student_data = student_data[~is_test_student].copy()

    except Exception as e:
        logging.error(f"An error occurred loading the main gradebook: {e}")
        return

    # --- 4. Consolidate ALL Source Files into the Master Record ---
    all_source_files = bc_filenames + ic_filenames
    processed_poll_columns = set()

    for filename in all_source_files:
        try:
            source_df = pd.read_csv(filename, dtype=str)
            source_students = source_df[pd.to_numeric(source_df['SIS User ID'], errors='coerce').notna()].copy()
            
            source_poll_col_name = next((col for col in source_students.columns if 'Class' in col and 'Poll' in col), None)
            if not source_poll_col_name: continue

            simple_poll_name = re.match(r'Class \d+ - Poll', source_poll_col_name).group(0)
            master_poll_col_name = poll_column_map.get(simple_poll_name, simple_poll_name)
            processed_poll_columns.add(master_poll_col_name)

            if master_poll_col_name not in student_data.columns:
                student_data[master_poll_col_name] = 0.0

            source_subset = source_students[['SIS User ID', source_poll_col_name]].rename(columns={source_poll_col_name: 'New_Score'})
            student_data = pd.merge(student_data, source_subset, on='SIS User ID', how='left')

            student_data[master_poll_col_name] = pd.to_numeric(student_data[master_poll_col_name], errors='coerce').fillna(0)
            student_data['New_Score'] = pd.to_numeric(student_data['New_Score'], errors='coerce').fillna(0)
            student_data[master_poll_col_name] = student_data[[master_poll_col_name, 'New_Score']].max(axis=1)
            
            student_data.drop(columns=['New_Score'], inplace=True)

        except Exception as e:
            logging.error(f"  - ❌ An error occurred while processing {filename}: {e}")
            continue

    # --- 5. Final Calculation and Reconstruction ---
    # Naturally sort the poll columns before summing and saving
    sorted_poll_columns = sorted(list(processed_poll_columns), key=natural_sort_key)
    logging.info(f"Final poll columns in order: {sorted_poll_columns}")

    logging.info(f"🔄 Recalculating '{master_total_col}' using all consolidated polls...")
    for col in sorted_poll_columns:
        student_data[col] = pd.to_numeric(student_data[col], errors='coerce').fillna(0)
    
    student_data[master_total_col] = student_data[sorted_poll_columns].sum(axis=1).round(2)
    
    final_df = pd.concat([special_rows, student_data], ignore_index=True)
    if not test_student_row.empty:
        final_df = pd.concat([final_df, test_student_row], ignore_index=True)

    # --- 6. Save Final Truncated File ---
    cols_to_keep = list(master_df.columns[:5]) + sorted_poll_columns + [master_total_col]
    cols_to_keep = [col for col in cols_to_keep if col in final_df.columns]
    
    final_df_truncated = final_df[cols_to_keep]

    # Move student test index to the last
    row_to_move = final_df_truncated.loc[[2]]
    final_df_truncated = pd.concat([final_df_truncated.drop(index=2), row_to_move], ignore_index=True)

    # First row to be empty
    final_df_truncated.iloc[0]=""
    
    
    output_filename = f'updated_{master_bc_filename}'
    final_df_truncated.to_csv(output_filename, encoding='utf-8-sig', index=False)
    
    logging.info(f"\n✨ Done! All files consolidated. Final gradebook saved to '{output_filename}'.")

    # --- 7. Eliminate all csv files -- 
    

# --- Main execution block ---
if __name__ == "__main__":
    run_final_grade_update()

INFO: ✅ Found 2 Canvas gradebooks and 2 iClicker files.
INFO: Built a map of 2 official poll column names.
INFO: Final poll columns in order: ['Class 1 - Poll (8935383)', 'Class 2 - Poll (8935377)']
INFO: 🔄 Recalculating 'iClicker (Total) (8935426)' using all consolidated polls...
/tmp/ipykernel_143/1867481245.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  final_df_truncated.iloc[0]=""
INFO: 
✨ Done! All files consolidated. Final gradebook saved to 'updated_2025-09-03T2032_Grades-CIVENG_93.csv'.
